In [ ]:
#import the dependencies


In [ ]:
#import the data as a dataframe

In [ ]:
#Select either the Department of Labor or the Department of Commerce dataframe and generate counts for each value and plot in bar charts along 
#with the count values either on the bar chart or a separate table.

In [ ]:
#Recode any variables where this is necessary. This must all be done in code because you will use the code to make the exact same changes to the second dataset later:
#

In [ ]:
#who does this predict poorly for
#how does it predict differently for 
#different sub-populations

#identify those it identifies correctly or incorrectly to 
#check for bias

#Key course parts for executing any project: Day 1 Supervised Learning: Activities 3 & 5; Day 2 Supervised Learning Activity 5; Day 2 Unsupervised Learning Activity 4